In [ ]:
# default_exp funcs.dataset

In [ ]:
%load_ext autoreload
%autoreload 2

# Dataset and transformations

In [ ]:
#export
from decode_fish.imports import *
from decode_fish.funcs.file_io import *
from typing import Union
import skimage.measure
from torch.utils.data import DataLoader
import random
from scipy.ndimage import gaussian_filter

In [ ]:
#export
class DecodeDataset:

    def __init__(self, path: Union[str, list],
                 dataset_tfms: list, 
                 rate_transform: list, 
                 bg_transform: list, 
                 num_iter: int = 5000, 
                 device: str = 'cpu'):
        """
        Basic Dataset

        Args:
            path (str): [image_path]
            dataset_tfms (list): transformation specific to dataset
            rate_transform ([type]): transformation for rate generation
            bg_transform ([type]): background transformation
            num_iter (int, optional):define lenth of dataset. Defaults to 5000.
            device (str, optional): device cpu or gpu]. Defaults to 'cpu'.
            scale ([type], optional): scaling image to value. Defaults to None.
            min_value ([type], optional): minimum value. Defaults to None.
        """
        
        self.image_path = glob.glob(path)
        self.dataset_tfms = dataset_tfms
        self.num_iter = num_iter
        self.local_rate_tfms = rate_transform
        self.bg_transform = bg_transform
        self.device = device
        
        self.volumes = [load_tiff_image(f) for f in self.image_path]
        
    def __len__(self):
        return self.num_iter

    def __getitem__(self, _):
        x = random.choice(self.volumes)
        x = self._compose(x, self.dataset_tfms)
        local_rate = self.local_rate_tfms(x)
        background = self.bg_transform(x)
        return x.to(self.device), local_rate.to(self.device), background.to(self.device)
    
    def __repr__(self):
        print (f'{self.__class__.__name__} Summary:')
        print (f'Dataset tfms: {len(self.dataset_tfms)}')
        for i in self.dataset_tfms:
            print (f'\n-->')
            f"{i}"
        return ''
    
    @staticmethod
    def _compose(x, list_func):
        if not list_func: list_func.append(lambda x: x)
        for func in list_func:x = func(x)
        return x

In [ ]:
#export
def print_class_signature(self, nms):
    "print class signature"
    mod = inspect.currentframe().f_back.f_locals
    for n in nms:
        print(f'{n}: {getattr(self,n)}')
#export
class TransformBase:
    '''
    All transformations optionally must be inherited from this class for nice
    representations and checks if input to given transformations is a tensor

    '''
    def __repr__(self):
        print (f'Transform({self.__class__.__name__})')
        name = inspect.signature(self.__class__).parameters.keys()
        print_clas_signature(self, name)
        return ''

    def __call__(self, x):
        assert isinstance(x, torch.Tensor), f'must be torch.tensor not {type(x)}'

    @staticmethod
    def _get_device(x):
        return getattr(x, 'device')

In [ ]:
#export
class ScaleTensor(TransformBase):
    """
    \nScales given `torch.Tensor` between `low` and `high`

    \nParameters:
    \n`low`     : lower bound
    \n`high`    : upper bound
    \n`data_min`: max value of data
    \n`data_max`: min value of main data

    \nReturns:
    \nScaled tensor

    """
    def __init__(self, low: float, high: float, data_min: float=0., data_max: float =1.):
        self.low = low
        self.high = high
        self.data_min = data_min
        self.data_max = data_max
        self.ratio = (self.high-self.low) /(self.data_max-self.data_min)

    def __call__(self, x) -> torch.Tensor:
        super().__call__(x)
        return self.ratio * x + self.low- self.data_min * self.ratio
    

class EstimateBackground(TransformBase):
    def __init__(self, smoothing_filter_size, div_factor=1):
        self.smoothing_filter_size = smoothing_filter_size
        self.div_factor = div_factor
    
    def __call__(self, image):
        background = gaussian_filter(image, self.smoothing_filter_size)/self.div_factor
        #background.clamp_min_(1.)
        return torch.tensor(background)
    
def get_forward_scaling(img):
    
    offset = np.median(img)
    scale = np.max(np.array(img))/3
    return offset, scale

In [ ]:
#export
class RandomCrop3D(TransformBase):
    """
    Ramdomly Crops 3D tensor.

    \nThis class will generate random crop of `crop_sz`. This calss is initilize
    with `img_sz` which should be a demension of 4 [Channel, Height, Width, Depth] and
    a `crop_sz` dimesnion of 3 [Height, Width, Depth] of desired crop. For each crop
    dimension `_get_slice` function will calculate random int ranging from 0 to (img_sz-crop_sz).
    and return tuple of containing two slice intergers. If one dimension of `img_sz` matches
    one dimension of `crop_sz` the resulting tuple will be `(None, None)` which will result
    in not croping this particular dimension.


    \nParameters:
    \n`img_sz`     : Size of the 3D image `(C, H, W, D)`
    \n`crop_sz`    : Size of the 3D crop  `(H, W, D)`

    \nReturns:
    \nCroped 3D image of the given `crop_sz`

    """
    def __init__(self, crop_sz, roi_mask):
        assert len(crop_sz) == 3 , f'Lenth of crop_sz should be 3 not {len(crop_sz)}'
        self.crop_sz = tuple(crop_sz)
        self.crop_prod = crop_sz[0]*crop_sz[1]*crop_sz[2]
        self.roi_mask = roi_mask

    def __call__(self, x):
        _, h, w, d = x.shape
        img_sz  = tuple((h, w, d))
        assert (img_sz) >  self.crop_sz
        super().__call__(x)
        slice_hwd = [self._get_slice(i, k) for i, k in zip(img_sz, self.crop_sz)]
        while self._crop(self.roi_mask[None], *slice_hwd).sum()/self.crop_prod < 0.5:
            slice_hwd = [self._get_slice(i, k) for i, k in zip(img_sz, self.crop_sz)]
        return self._crop(x, *slice_hwd)


    @staticmethod
    def _get_slice(sz, crop_sz):
        up_bound = sz-crop_sz
        if  up_bound == 0:
            return None, None
        else:
            l_bound = torch.randint(up_bound, (1,))
        return l_bound, l_bound + crop_sz

    @staticmethod
    def _crop(x, slice_h, slice_w, slice_d):
        return x[:, slice_h[0]:slice_h[1], slice_w[0]:slice_w[1], slice_d[0]:slice_d[1]]

In [ ]:
#export
def get_roi_mask(img, pool_size=(10,10,10), percentile=50):
    mp_img = skimage.measure.block_reduce(img, pool_size, np.max)
    thresh = np.percentile(mp_img,percentile)
    mp_img = np.where(mp_img>thresh,1,0)
    mp_img = np.kron(mp_img, np.ones((pool_size), dtype=mp_img.dtype))
    mp_img = mp_img[:img.shape[0], :img.shape[1], :img.shape[2]] 
    return mp_img

In [ ]:
from decode_fish.funcs.file_io import load_psf_noise_micro
from decode_fish.engine.point_process import PointProcessUniform
from decode_fish.funcs.plotting import *

In [ ]:
img_3d = load_tiff_image('/groups/turaga/home/speisera/share_TUM/FishSIM/sim_1/mRNAlevel_200/cell3D/strong/w1_HelaKyoto_Gapdh_2597_p01_cy3__Cell_CP_14__cell3D__1.tif')[0]
estimate_backg = EstimateBackground(smoothing_filter_size=6, div_factor=1)
roi_mask       = get_roi_mask(img_3d, pool_size=(10,10,10), percentile=70)
rand_crop      = RandomCrop3D((48,48,48), roi_mask)

In [ ]:
probmap_generator = ScaleTensor(low=0.00000001, 
                                high=0.005, 
                                data_min = img_3d.min(), 
                                data_max = img_3d.max())

ds = DecodeDataset(path = '/groups/turaga/home/speisera/share_TUM/FishSIM/sim_1/mRNAlevel_200/cell3D/strong/w1_HelaKyoto_Gapdh_2597_p01_cy3__Cell_CP_14__cell3D__1.tif',
                   dataset_tfms =  [rand_crop], 
                   rate_transform = probmap_generator, 
                   bg_transform = estimate_backg, 
                   device='cuda:0', 
                   num_iter=100 * 4) 

In [ ]:
decode_dl = DataLoader(ds, batch_size=4, num_workers=0)

In [ ]:
x, local_rate, background = next(iter(decode_dl))

/groups/turaga/home/speisera/anaconda3/envs/decode2_dev/lib/python3.7/site-packages/torch/cuda/__init__.py:81: UserWarning: 
    Found GPU2 NVS 510 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [ ]:
cfg = OmegaConf.load(default_conf)

In [ ]:
psf, noise, micro = load_psf_noise_micro(cfg)

In [ ]:
#export
def get_dataloader(cfg):
    
    img_3d = load_tiff_image(cfg.data_path.image_path)[0]
    estimate_backg = hydra.utils.instantiate(cfg.bg_estimation)
    roi_mask       = get_roi_mask(img_3d, tuple(cfg.roi_mask.pool_size), percentile= cfg.roi_mask.percentile)
    rand_crop      = RandomCrop3D((cfg.random_crop.crop_sz,cfg.random_crop.crop_sz,cfg.random_crop.crop_sz), roi_mask)
    
    probmap_generator = ScaleTensor(low=cfg.prob_generator.low, 
                                    high=cfg.prob_generator.high, 
                                    data_min = img_3d.min(), 
                                    data_max = img_3d.max())

    ds = DecodeDataset(path = cfg.data_path.image_path,
                       dataset_tfms =  [rand_crop], 
                       rate_transform = probmap_generator, 
                       bg_transform = estimate_backg, 
                       device='cuda:0', 
                       num_iter=cfg.dataloader.num_iter * cfg.dataloader.bs) 

    decode_dl = DataLoader(ds, batch_size=cfg.dataloader.bs, num_workers=0)
    decode_dl.min_int = cfg.pointprocess.min_int
    
    return img_3d, decode_dl

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted index.ipynb.
